In [9]:
# ------------------------------------------------------------
# ASC Email Composer — Portable Edition
# Author: Nana Kwaku Amoako
# GitHub: https://github.com/nanadotam/
# Date: July 2025
# Description:
# This interactive tool allows you to compose, preview, and send
# branded emails with embedded images and file attachments using
# markdown-style formatting.
# ------------------------------------------------------------

# FOLLOW ALL INSTRUCTIONS.
# THIS WORKS BEST LOCALLY. DOWNLOAD AND RUN IN JUPYTER NOTEBOOKS!!!
## DO NOT RUN ALL THE CODE CHUNKS AT ONCE. DO IT STEP BY STEP

### 🚨 KNOWN ISSUES
1.When running in Colab, Microsoft bounces emails due to a security alert triggered by the servers being in a different location from the user.

✅ Known Fix: Run locally as the IP address of your local machine will match the ones from the Outlook on your machine

In [1]:
!pip install ipywidgets

# 📤 Step 1: File Upload + Column Mapping + Email Validation

In [ ]:
import pandas as pd
import re
from ipywidgets import FileUpload, Dropdown, VBox, Output, Label, Button, HTML, HBox
from IPython.display import display, clear_output

# Widgets
upload_widget = FileUpload(accept='.csv', multiple=False)
output_preview = Output()
column_selector_name = Dropdown(description="Name column")
column_selector_email = Dropdown(description="Email column")
submit_columns_btn = Button(description="✅ Confirm Columns", button_style='success')
validation_output = Output()

# Global DataFrame
df = pd.DataFrame()

def handle_upload(change):
  clear_output(wait=True)
  output_preview.clear_output()
  global df
  if upload_widget.value:
      try:
          file_info = list(upload_widget.value.values())[0]
          content = file_info['content']
          df = pd.read_csv(pd.io.common.BytesIO(content))

          with output_preview:
              print("✅ CSV File Uploaded Successfully!")
              print(f"📊 Found {len(df)} rows and {len(df.columns)} columns")

              # Auto-detect likely name and email columns
              name_candidates = [col for col in df.columns if any(word in col.lower() for word in ['name', 'full', 'first', 'last'])]
              email_candidates = [col for col in df.columns if 'email' in col.lower() or 'mail' in col.lower()]

              if name_candidates:
                  print(f"💡 Auto-detected Name column: '{name_candidates[0]}'")
              if email_candidates:
                  print(f"💡 Auto-detected Email column: '{email_candidates[0]}'")
              if not name_candidates or not email_candidates:
                  print("⚠️ Couldn't auto-detect name/email columns. Please select manually below.")

              print("\n👀 Preview of your data:")
              display(df.head())

              print("\n📝 Available placeholders for your email template:")
              for col in df.columns:
                  print(f"   {{{{ {col} }}}}")
              print("\n💡 Copy these exact placeholder names to use in Step 2!")

          # Update dropdown options
          columns = df.columns.tolist()
          column_selector_name.options = columns
          column_selector_email.options = columns

          # Auto-select if we found candidates
          if name_candidates:
              column_selector_name.value = name_candidates[0]
          if email_candidates:
              column_selector_email.value = email_candidates[0]

          display(VBox([
              output_preview,
              HTML(value="<hr style='border: 2px solid #007acc; margin: 20px 0;'>"),
              HTML(value="<h3 style='color: #007acc;'>📋 Step 1.2: Select Your Columns</h3>"),
              HTML(value="<p style='color: #666; margin: 10px 0;'>Choose which columns contain the name and email addresses for your recipients:</p>"),
              HBox([column_selector_name, column_selector_email]),
              submit_columns_btn,
              validation_output
          ]))

      except Exception as e:
          with output_preview:
              print("❌ Error reading CSV file!")
              print(f"Error details: {str(e)}")
              print("\n💡 Troubleshooting steps:")
              print("   ✓ Make sure your file is a valid CSV format")
              print("   ✓ Ensure the file has column headers in the first row")
              print("   ✓ Check that the file isn't corrupted or empty")
              print("   ✓ Try saving your Excel file as CSV if needed")

  # Detect available placeholders from columns
  if not df.empty:
      placeholder_candidates = [col for col in df.columns if col not in [column_selector_name.value, column_selector_email.value]]
      st_placeholders = placeholder_candidates  # Save for Step 2 usage


def validate_email_list(name_col, email_col):
  validation_output.clear_output()
  if df.empty:
      return
  errors = []
  for idx, row in df.iterrows():
      email = str(row[email_col]).strip()
      name = str(row[name_col]).strip()
      if not re.match(r"[^@]+@[^@]+\.[^@]+", email):
          errors.append((idx+1, name, email))
  with validation_output:
      if errors:
          print("❌ Invalid email addresses found:")
          for row_num, name, email in errors:
              print(f"Row {row_num}: {name} - {email}")
      else:
          print("✅ All emails are valid!")

submit_columns_btn.on_click(lambda b: validate_email_list(
    column_selector_name.value, column_selector_email.value
))
upload_widget.observe(handle_upload, names='value')

# Display upload UI with enhanced guidance
display(HTML(value="""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
<h2 style="margin: 0; text-align: center;">📧 Personal - ASC SMTP Email Service - Step 1</h2>
<p style="text-align: center; margin: 5px 0;">Upload your contact list to get started</p>
</div>
"""))

display(HTML(value="""
<div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #007acc;">
<h3 style="color: #007acc; margin-top: 0;">📋 CSV File Requirements</h3>
<ul style="margin: 10px 0;">
<li><strong>Format:</strong> Must be a .csv file (not .xlsx or .xls)</li>
<li><strong>Headers:</strong> First row should contain column names</li>
<li><strong>Required columns:</strong> At least one for names and one for email addresses</li>
<li><strong>Example format:</strong></li>
</ul>
<pre style="background: #e9ecef; padding: 10px; border-radius: 4px; font-family: monospace;">
Name,Email,Event,Location
John Doe,john@example.com,Conference,New York
Jane Smith,jane@example.com,Conference,New York
</pre>
</div>
"""))

display(HTML(value="<h3 style='color: #007acc;'>📤 Step 1.1: Upload Your CSV File</h3>"))
display(upload_widget)


HTML(value='\n<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding…

HTML(value='\n<div style="background: #f8f9fa; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

HTML(value="<h3 style='color: #007acc;'>📤 Step 1.1: Upload Your CSV File</h3>")

FileUpload(value=(), accept='.csv', description='Upload')


✅ Step 1 is ready. After uploading CSV, proceed to Step 2.


# 📥 📨 Step 2 – Styled Email Composer with Preview + Test Send

In [2]:
import smtplib
import os
import re
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from ipywidgets import Text, Textarea, Button, Output, HTML, VBox, HBox, Layout, Password
from IPython.display import display, clear_output

# ====================================================
# STEP 2: HTML EMAIL TEMPLATE SYSTEM
# ====================================================

# Import global variables from Step 1
# Check if they exist, if not create dummy versions
try:
    # Test if df exists from Step 1
    test_df = df
    test_name = column_selector_name
    test_email = column_selector_email
except NameError:
    # If variables don't exist, they'll be available after Step 1 runs
    print("⚠️ Note: Please run Step 1 first to upload your CSV file")

# --- Input Widgets ---
sender_email_input = Text(
    value='',
    placeholder='yourname@example.com',
    description='Email:',
    layout=Layout(width='100%')
)

password_input = Password(
    value='',
    placeholder='Your email password',
    description='Password:',
    layout=Layout(width='100%')
)

subject_input = Text(
    value='✨ Ubora 2025 – Your Night of Excellence Awaits',
    placeholder='Email subject line',
    description='Subject:',
    layout=Layout(width='100%')
)

test_email_input = Text(
    value='',
    placeholder='test@example.com',
    description='Test Email:',
    layout=Layout(width='100%')
)

# --- Output and Button Widgets ---
preview_output = Output()
test_send_output = Output()
preview_button = Button(description="👁️ Preview Email", button_style='info')
test_send_button = Button(description="📧 Send Test Email", button_style='success')

# ====================================================
# LOAD HTML TEMPLATE
# ====================================================
def load_html_template():
    """Load the HTML template from email.html"""
    try:
        with open('email.html', 'r', encoding='utf-8') as f:
            return f.read()
    except FileNotFoundError:
        return "<html><body><h1>Error: email.html not found!</h1><p>Please ensure email.html is in the same directory as this notebook.</p></body></html>"
    except Exception as e:
        return f"<html><body><h1>Error loading template:</h1><p>{str(e)}</p></body></html>"

# ====================================================
# PERSONALIZE HTML WITH ROW DATA
# ====================================================
def generate_email_html(row):
    """
    Load HTML template and replace {Name} placeholder with actual name from CSV row.
    You can add more placeholders here if needed.
    """
    html_template = load_html_template()
    
    # Get the name from the row
    try:
        # Use globals() to access variables from Step 1
        name_col = globals()['column_selector_name'].value
        name = str(row[name_col]).strip()
    except:
        name = "Friend"
    
    # Replace {Name} placeholder in the HTML
    personalized_html = html_template.replace("{Name}", name)
    
    # Add more replacements here if your CSV has other columns you want to use
    # Example: personalized_html = personalized_html.replace("{Event}", row.get('Event', 'Ubora'))
    
    return personalized_html

# ====================================================
# PREVIEW FUNCTION
# ====================================================
def preview_email(b):
    preview_output.clear_output()
    
    with preview_output:
        # Check if df exists
        try:
            current_df = globals()['df']
            current_name_col = globals()['column_selector_name']
            current_email_col = globals()['column_selector_email']
        except KeyError:
            print("❌ No CSV loaded. Upload a CSV in Step 1 first!")
            print("💡 Make sure you've run Step 1 and uploaded a CSV file")
            return
        
        if current_df.empty:
            print("❌ CSV appears empty. Please upload a valid CSV in Step 1!")
            return
        
        print("📧 Email Preview")
        print("="*50)
        print(f"From: {sender_email_input.value}")
        print(f"Subject: {subject_input.value}")
        print("="*50)
        
        # Generate preview using first row
        first_row = current_df.iloc[0]
        html = generate_email_html(first_row)
        
        # Display HTML preview
        print("\n🎨 HTML Email Preview (with first recipient's data):")
        print(f"Recipient Name: {first_row[current_name_col.value]}")
        print(f"Recipient Email: {first_row[current_email_col.value]}")
        print("\n")
        display(HTML(value=html))
        
        # Check if images folder exists
        if os.path.isdir('images'):
            image_files = [f for f in os.listdir('images') if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]
            print(f"\n✅ Found {len(image_files)} images in /images folder")
            print("Images that will be embedded:")
            for img in image_files:
                print(f"   • {img}")
        else:
            print("\n⚠️ Warning: 'images' folder not found!")
            print("   Make sure the 'images' folder is in the same directory as this notebook")

# ====================================================
# TEST SEND FUNCTION
# ====================================================
def send_test_email(b):
    test_send_output.clear_output()
    
    with test_send_output:
        # Check if df exists
        try:
            current_df = globals()['df']
            current_name_col = globals()['column_selector_name']
            current_email_col = globals()['column_selector_email']
        except KeyError:
            print("❌ No CSV loaded. Upload a CSV in Step 1 first!")
            print("💡 Make sure you've run Step 1 and uploaded a CSV file")
            return
        
        if current_df.empty:
            print("❌ CSV appears empty. Please upload a valid CSV in Step 1!")
            return
        
        if not sender_email_input.value or not password_input.value:
            print("❌ Please enter your email and password first!")
            return
        
        if not test_email_input.value:
            print("❌ Please enter a test email address!")
            return
        
        if not re.match(r"[^@]+@[^@]+\.[^@]+", test_email_input.value):
            print("❌ Invalid test email address format!")
            return
        
        print("📧 Sending test email...")
        print(f"From: {sender_email_input.value}")
        print(f"To: {test_email_input.value}")
        print(f"Subject: {subject_input.value}")
        
        try:
            # Create message with first row data
            first_row = current_df.iloc[0]
            
            msg = MIMEMultipart("related")
            msg["Subject"] = subject_input.value
            msg["From"] = sender_email_input.value
            msg["To"] = test_email_input.value
            
            # Generate HTML
            html = generate_email_html(first_row)
            alt_part = MIMEMultipart("alternative")
            alt_part.attach(MIMEText(html, "html"))
            msg.attach(alt_part)
            
            # Attach images
            image_folder = "images"
            if os.path.isdir(image_folder):
                image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg', '.gif'))]
                for img_name in image_files:
                    img_path = os.path.join(image_folder, img_name)
                    try:
                        with open(img_path, "rb") as f:
                            img_data = f.read()
                            img = MIMEImage(img_data)
                            img.add_header("Content-ID", f"<{img_name}>")
                            img.add_header("Content-Disposition", "inline", filename=img_name)
                            msg.attach(img)
                    except Exception as e:
                        print(f"⚠️ Warning: Could not attach image {img_name}: {e}")
                
                print(f"✅ Attached {len(image_files)} images")
            else:
                print("⚠️ Warning: 'images' folder not found - sending without images")
            
            # Connect and send
            print("\n🔗 Connecting to SMTP server...")
            server = smtplib.SMTP("smtp.office365.com", 587)
            server.starttls()
            server.login(sender_email_input.value, password_input.value)
            
            print("📤 Sending...")
            server.sendmail(sender_email_input.value, test_email_input.value, msg.as_string())
            server.quit()
            
            print("\n✅ TEST EMAIL SENT SUCCESSFULLY!")
            print(f"Check {test_email_input.value} inbox")
            print("\n💡 If the email looks good, proceed to Step 3 to send to all recipients")
            
        except smtplib.SMTPAuthenticationError:
            print("\n❌ Authentication failed!")
            print("   • Check your email address and password")
            print("   • If using 2FA, you may need an app-specific password")
            print("   • Visit: https://support.microsoft.com/en-us/account-billing/using-app-passwords-with-apps-that-don-t-support-two-step-verification-5896ed9b-4263-e681-128a-a6f2979a7944")
        except Exception as e:
            print(f"\n❌ Error sending test email: {str(e)}")

# --- Connect buttons ---
preview_button.on_click(preview_email)
test_send_button.on_click(send_test_email)

# ====================================================
# DISPLAY UI
# ====================================================
display(HTML(value='<h2>ASC SMTP — Ubora HTML Email System</h2>'))

display(HTML(value="""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
<h2 style="margin: 0; text-align: center;">📨 Step 2: Configure & Preview Your Email</h2>
<p style="text-align: center; margin: 5px 0;">Using email.html template with embedded images</p>
</div>
"""))

display(HTML(value="""
<div style="background: #d1ecf1; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #0c5460;">
<h3 style="color: #0c5460; margin-top: 0;">📋 Template Information</h3>
<ul style="margin: 10px 0; color: #0c5460;">
<li><strong>Template:</strong> email.html (Ubora event invitation)</li>
<li><strong>Images:</strong> Automatically loaded from /images folder</li>
<li><strong>Personalization:</strong> {Name} placeholder will be replaced with each recipient's name</li>
<li><strong>Subject:</strong> Customize the subject line below</li>
</ul>
</div>
"""))

display(VBox([
    HTML(value="<h3 style='color: #007acc;'>🔐 Step 2.1: Email Credentials</h3>"),
    sender_email_input,
    password_input,
    HTML(value="<p style='color: #666; font-size: 0.9em; margin: 5px 0;'>💡 Your password is not stored and only used for sending emails</p>"),
]))

display(VBox([
    HTML(value="<h3 style='color: #007acc;'>📝 Step 2.2: Email Subject</h3>"),
    subject_input,
]))

display(VBox([
    HTML(value="<h3 style='color: #007acc;'>👁️ Step 2.3: Preview Your Email</h3>"),
    HTML(value="<p style='color: #666; margin: 5px 0;'>See how your email will look (using data from the first recipient):</p>"),
    preview_button,
    preview_output,
]))

display(VBox([
    HTML(value="<hr style='border: 1px solid #dee2e6; margin: 20px 0;'>"),
    HTML(value="<h3 style='color: #28a745;'>📧 Step 2.4: Send Test Email</h3>"),
    HTML(value="<p style='color: #666; margin: 5px 0;'>Send a test email to yourself before sending to all recipients:</p>"),
    test_email_input,
    test_send_button,
    test_send_output,
]))

display(HTML(value="""
<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #ffc107;">
<h3 style="color: #856404; margin-top: 0;">⚠️ Important Notes</h3>
<ul style="margin: 10px 0; color: #856404;">
<li>The HTML template is loaded from <strong>email.html</strong></li>
<li>All images in the <strong>/images</strong> folder will be embedded automatically</li>
<li><strong>{Name}</strong> in the template will be replaced with each recipient's name from your CSV</li>
<li>Always send a test email before proceeding to Step 3</li>
<li>If using Outlook/Microsoft 365 with 2FA, you'll need an app-specific password</li>
</ul>
</div>
"""))


HTML(value='<h2>ASC SMTP — Ubora HTML Email System</h2>')

HTML(value='\n<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding…

HTML(value='\n<div style="background: #d1ecf1; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

HTML(value='\n<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

# 📤 Step 3: Final Checks & Bulk Send

In [11]:
from tqdm.notebook import tqdm
import time
import smtplib
import re
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from ipywidgets import Button, Output, HTML, VBox, HBox
from IPython.display import display
import os

# --- Send Button + Output Widgets ---
send_all_button = Button(description="🚀 Send All Emails", button_style='danger')
stop_button = Button(description="⏹️ Stop Sending", button_style='warning', disabled=True)
confirm_button = Button(description="⚠️ Confirm Send to ALL Recipients", button_style='danger', disabled=True)
cancel_button = Button(description="❌ Cancel", button_style='')
send_output = Output()
confirmation_output = Output()

# --- Global flags ---
stop_sending = False
confirmed = False

# --- Confirmation Functions ---
def show_confirmation(b):
    global confirmed
    confirmed = False
    confirmation_output.clear_output()

    if df.empty:
        with confirmation_output:
            print("❌ No CSV data loaded. Please complete Step 1 first.")
        return

    recipient_count = len(df)

    with confirmation_output:
        print(f"⚠️ CONFIRMATION REQUIRED")
        print(f"You're about to send {recipient_count} emails!")
        print(f"📧 From: {sender_email_input.value}")
        print(f"📝 Subject: {subject_input.value}")
        print(f"👥 Recipients: {recipient_count}")

        if recipient_count > 50:
            print("\n🚨 WARNING: Large recipient list detected!")
            print("   Consider breaking this into smaller batches")

        print("\n💡 This action cannot be undone!")
        print("   Make sure you've tested your email first")
        print("   Double-check your recipient list")

    send_all_button.disabled = True
    confirm_button.disabled = False
    cancel_button.disabled = False

def confirm_send(b):
    global confirmed
    confirmed = True
    confirmation_output.clear_output()

    with confirmation_output:
        print("✅ Confirmed! Starting bulk email send...")

    confirm_button.disabled = True
    cancel_button.disabled = True
    send_bulk_emails(None)

def cancel_send(b):
    global confirmed
    confirmed = False
    confirmation_output.clear_output()

    with confirmation_output:
        print("❌ Send cancelled.")

    send_all_button.disabled = False
    confirm_button.disabled = True
    cancel_button.disabled = True

# --- Stop Function ---
def stop_sending_emails(b):
    global stop_sending
    stop_sending = True
    with send_output:
        print("\n🛑 Stop requested...")

# ====================================================
# UPDATED BULK EMAIL FUNCTION WITH CID IMAGE SUPPORT
# ====================================================
def send_bulk_emails(b):
    global stop_sending
    stop_sending = False

    send_output.clear_output()
    if df.empty:
        with send_output:
            print("❌ No CSV loaded.")
        return

    if not sender_email_input.value or not password_input.value:
        with send_output:
            print("❌ Enter email + password first.")
        return

    send_all_button.disabled = True
    confirm_button.disabled = True
    cancel_button.disabled = True
    stop_button.disabled = False

    name_col = column_selector_name.value
    email_col = column_selector_email.value
    total_recipients = len(df)

    failed_list = []
    sent_count = 0
    start_time = time.time()

    try:
        with send_output:
            print("🔗 Connecting to server...")

        server = smtplib.SMTP("smtp.office365.com", 587)
        server.starttls()
        server.login(sender_email_input.value, password_input.value)

        with send_output:
            print("✅ Connected!")
            print(f"📧 From: {sender_email_input.value}")
            print(f"📝 Subject: {subject_input.value}")
            print("📨 Starting send...\n")

        for index, row in tqdm(df.iterrows(), total=len(df), desc="Sending emails"):
            if stop_sending:
                with send_output:
                    print(f"\n🛑 Stopped at {sent_count}/{total_recipients}.")
                break

            name = str(row[name_col]).strip()
            recipient = str(row[email_col]).strip()

            if not re.match(r"[^@]+@[^@]+\.[^@]+", recipient):
                failed_list.append((name, recipient, "Invalid email"))
                continue

            try:
                # ====================================================
                # NEW: Create a "related" email to embed inline images
                # ====================================================
                msg = MIMEMultipart("related")
                msg["Subject"] = subject_input.value
                msg["From"] = sender_email_input.value
                msg["To"] = recipient

                html = generate_email_html(row)
                alt_part = MIMEMultipart("alternative")
                alt_part.attach(MIMEText(html, "html"))
                msg.attach(alt_part)

                # ====================================================
                # NEW: Attach images from /images with Content-ID
                # ====================================================
                image_folder = "images"
                if os.path.isdir(image_folder):
                    for img_name in os.listdir(image_folder):
                        img_path = os.path.join(image_folder, img_name)
                        try:
                            with open(img_path, "rb") as f:
                                img = MIMEImage(f.read())
                                img.add_header("Content-ID", f"<{img_name}>")
                                img.add_header("Content-Disposition", "inline", filename=img_name)
                                msg.attach(img)
                        except Exception as e:
                            failed_list.append((name, recipient, f"Failed image: {img_name}"))
                            continue

                # Send
                server.sendmail(sender_email_input.value, recipient, msg.as_string())
                sent_count += 1

                if sent_count % 10 == 0:
                    elapsed = time.time() - start_time
                    avg = elapsed / sent_count
                    remaining = (total_recipients - sent_count) * avg

                    with send_output:
                        print(f"📈 {sent_count}/{total_recipients} sent ({sent_count/total_recipients*100:.1f}%)")
                        print(f"⏱️ Remaining ~ {remaining/60:.1f} minutes")

                time.sleep(0.5)

            except Exception as e:
                failed_list.append((name, recipient, f"Error: {e}"))
                continue

        server.quit()

        elapsed = time.time() - start_time
        success_rate = (sent_count / total_recipients) * 100

        with send_output:
            print("\n" + "="*50)
            print("📊 CAMPAIGN SUMMARY")
            print("="*50)
            print(f"✅ Sent: {sent_count}/{total_recipients} ({success_rate:.1f}%)")
            print(f"⏱️ Total time: {elapsed/60:.1f} min")

            if failed_list:
                print(f"\n❌ Failed: {len(failed_list)}")
                for entry in failed_list:
                    print(f" - {entry[0]} ({entry[1]}): {entry[2]}")

            if sent_count == total_recipients and not stop_sending:
                print("\n🎉 ALL EMAILS SENT SUCCESSFULLY!")

    except smtplib.SMTPAuthenticationError:
        with send_output:
            print("❌ Auth failed. Check email + password.")
    except Exception as e:
        with send_output:
            print(f"🚨 Unexpected error: {e}")

    finally:
        send_all_button.disabled = False
        stop_button.disabled = True
        confirm_button.disabled = True
        cancel_button.disabled = True

# --- Hook Up Buttons ---
send_all_button.on_click(show_confirmation)
stop_button.on_click(stop_sending_emails)
confirm_button.on_click(confirm_send)
cancel_button.on_click(cancel_send)

# --- Display Enhanced UI ---
display(HTML(value="""
<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding: 20px; border-radius: 10px; margin: 10px 0;">
<h2 style="margin: 0; text-align: center;">🚀 Personal SMTP Email Service - Step 3</h2>
<p style="text-align: center; margin: 5px 0;">Send your emails to all recipients</p>
</div>
"""))

display(HTML(value="""
<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0; border-left: 4px solid #ffc107;">
<h3 style="color: #856404; margin-top: 0;">⚠️ Safety Guidelines</h3>
<ul style="margin: 10px 0; color: #856404;">
<li><strong>Test First:</strong> Always send a test email before bulk sending</li>
<li><strong>Check Recipients:</strong> Ensure your CSV is correct</li>
<li><strong>Small Batches:</strong> For 50+ recipients, break into batches</li>
<li><strong>Monitor:</strong> Use the stop button if needed</li>
<li><strong>Backup:</strong> Keep failed emails for retry</li>
</ul>
</div>
"""))

display(VBox([
    HTML(value="<h3 style='color: #dc3545;'>🚀 Bulk Email Sending</h3>"),
    HTML(value="<p style='color: #666; margin: 10px 0;'>Start the process:</p>"),
    HBox([send_all_button, stop_button]),
    confirmation_output,
    HTML(value="<h4 style='color: #dc3545;'>Confirmation Required</h4>"),
    HBox([confirm_button, cancel_button]),
    HTML(value="<hr style='border: 1px solid #dee2e6; margin: 20px 0;'>"),
    HTML(value="<h4 style='color: #007bff;'>📊 Sending Progress & Results</h4>"),
    send_output
]))


HTML(value='\n<div style="background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; padding…

HTML(value='\n<div style="background: #fff3cd; padding: 15px; border-radius: 8px; margin: 10px 0; border-left:…

In [12]:
# =============================================
# 🛠️ Crafted with care and ❤️ by Nana Amoako | July 2025
# For: Personalized ASC Email Campaign Project
# Features:
#  → Markdown-based email editor
#  → Embedded images + attachments
#  → Real-time preview + test email support
# =============================================